<a href="https://colab.research.google.com/github/YIBoX-memorial/ComputationalNeuroscience/blob/master/hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import scipy.io
from math import exp
from math import log
from math import cos
from math import sin
from math import pi
from math import ceil
from math import sqrt
from math import factorial
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
! nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



# Load Mat

In [2]:
input = scipy.io.loadmat('ps3_realdata.mat')
class load_mat:
    def __init__(self, input):
        self.train_trial = input['train_trial']
        self.test_trial = input['test_trial']
    def _check(self):
        print('train_trial: ', self.train_trial.shape, 'test_trial: ', self.test_trial.shape)
        '''
        for item in self.train_trial:
            for item2 in item:
                print(item2[1][0])
        '''

loader = load_mat(input)
loader._check()

train_trial:  (91, 8) test_trial:  (91, 8)


# Get $\Sigma$

In [32]:
def get_square(input):
    if input.ndim == 1:
        input = np.expand_dims(input, axis=1)
    return np.asmatrix(input) * np.asmatrix(input).reshape(input.shape[0])

def normal_prob(data, mean, variance):
    if data.ndim == 1:
        data = np.expand_dims(data, axis=0)
    if mean.ndim == 1:
        mean = np.expand_dims(mean, axis=0)
    # print(data.shape, variance.shape, mean.shape)
    return -0.5*log(np.linalg.det(variance)) - 0.5*np.asmatrix(data-mean)*np.asmatrix(variance).getI()*np.transpose(np.asmatrix(data-mean))

def poisson_prob(data, lam):
    fac = 0.0
    data = int(data)
    for i in range(1, data+1):
        fac = fac + log(i)
    if lam == 0.0 and data == 0:
        return 1
    if lam == 0.0 and data > 0:
        return 0
    return data*log(lam) - lam - fac
'''
a = np.asarray([[1],[2],[3],[4]])
b = np.asarray([[4],[3],[2],[1]])
b = np.asmatrix(b) + a
print(a.shape)
print(get_square(a))
'''

'\na = np.asarray([[1],[2],[3],[4]])\nb = np.asarray([[4],[3],[2],[1]])\nb = np.asmatrix(b) + a\nprint(a.shape)\nprint(get_square(a))\n'

# Porcessor of model 1

In [32]:
class P1:
    def __init__(self, train, test):
        self.train = train
        self.test = test
        super().__init__()
    def _pre_train_proc(self):
        self.trials = self.train.shape[0]
        self.angles = self.train.shape[1]
        self.neurons = self.train[0][0][1].shape[0]
        self.beg = 351
        self.end = 551
    def _train_proc(self):
        self.mean = np.zeros((self.angles, self.neurons), dtype=np.float)
        self.variance = np.zeros((self.angles, self.neurons, self.neurons), dtype=np.float)
        for (trial_id, trial) in enumerate(self.train):
            for (angle_id, angle) in enumerate(trial):
                # print(self.mean[angle_id].shape, angle[1].sum(1).shape)
                self.mean[angle_id] = self.mean[angle_id] + angle[1][:,self.beg:self.end].sum(1)
                # print(angle[0][0], angle[1][:,self.beg:self.end].sum())
        self.mean = self.mean / self.trials
        for (trial_id, trial) in enumerate(self.train):
            for (angle_id, angle) in enumerate(trial):
                # print((angle[1][:,self.beg:self.end].sum(1)-self.mean[angle_id]).shape)
                # print(get_square(angle[1][:,self.beg:self.end].sum(1)-self.mean[angle_id]))
                self.variance[angle_id] = self.variance[angle_id] + get_square(angle[1][:,self.beg:self.end].sum(1)-self.mean[angle_id])
        self.variance = self.variance / (self.trials*self.angles)
        self.variance = np.sum(self.variance, axis=0)
        print(self.mean.shape, self.variance.shape)
    def _pre_test_proc(self):
        self.trials = self.test.shape[0]
        self.angles = self.test.shape[1]
        self.neurons = self.test[0][0][1].shape[0]
        self.beg = 351
        self.end = 551
    def _test_proc(self):
        accuracy_cases = 0
        for (trial_id, trial) in enumerate(self.test):
            for (angle_id, angle) in enumerate(trial):
                max_value = -100000
                max_id = 0
                for (comp_id, mean) in enumerate(self.mean):
                    temp = normal_prob(angle[1][:,self.beg:self.end].sum(1), mean, self.variance)
                    if temp > max_value:
                        max_value = temp
                        max_id = comp_id
                if max_id == angle_id:
                    accuracy_cases = accuracy_cases + 1
        accuracy_rate = accuracy_cases / (self.trials*self.angles)
        return accuracy_rate

proc1 = P1(input['train_trial'], input['test_trial'])
proc1._pre_train_proc()
proc1._train_proc()
proc1._pre_test_proc()
acr = proc1._test_proc()
print(acr)

(8, 97) (97, 97)
0.9574175824175825


# Processor of model 2

In [45]:
class P2:
    def __init__(self, train, test):
        self.train = train
        self.test = test
        super().__init__()
    def _pre_train_proc(self):
        self.trials = self.train.shape[0]
        self.angles = self.train.shape[1]
        self.neurons = self.train[0][0][1].shape[0]
        self.beg = 351
        self.end = 551
    def _train_proc(self):
        self.mean = np.zeros((self.angles, self.neurons), dtype=np.float)
        self.variance = np.zeros((self.angles, self.neurons, self.neurons), dtype=np.float)
        for (trial_id, trial) in enumerate(self.train):
            for (angle_id, angle) in enumerate(trial):
                # print(self.mean[angle_id].shape, angle[1].sum(1).shape)
                self.mean[angle_id] = self.mean[angle_id] + angle[1][:,self.beg:self.end].sum(1)
                # print(angle[0][0], angle[1][:,self.beg:self.end].sum())
        self.mean = self.mean / self.trials
        for (trial_id, trial) in enumerate(self.train):
            for (angle_id, angle) in enumerate(trial):
                # print((angle[1][:,self.beg:self.end].sum(1)-self.mean[angle_id]).shape)
                # print(get_square(angle[1][:,self.beg:self.end].sum(1)-self.mean[angle_id]))
                self.variance[angle_id] = self.variance[angle_id] + get_square(angle[1][:,self.beg:self.end].sum(1)-self.mean[angle_id])
        self.variance = self.variance / self.trials
        print(self.mean.shape, self.variance.shape)
    def _pre_test_proc(self):
        self.trials = self.test.shape[0]
        self.angles = self.test.shape[1]
        self.neurons = self.test[0][0][1].shape[0]
        self.beg = 351
        self.end = 551
    def _test_proc(self):
        accuracy_cases = 0
        for (trial_id, trial) in enumerate(self.test):
            for (angle_id, angle) in enumerate(trial):
                max_value = -10000
                max_id = 0
                for comp_id in range(self.angles):
                    print(angle[1][:,self.beg:self.end].sum(1).shape, self.mean[comp_id].shape, self.variance[comp_id].shape)
                    print(np.linalg.det(np.asmatrix(self.variance[comp_id])))
                    print(np.asmatrix(self.variance[comp_id]).getI())
                    temp = normal_prob(angle[1][:,self.beg:self.end].sum(1), self.mean[comp_id], self.variance[comp_id])
                    if temp > max_value:
                        max_value = temp
                        max_id = comp_id
                if max_id == angle_id:
                    accuracy_cases = accuracy_cases + 1
        accuracy_rate = accuracy_cases / (self.trials*self.angles)
        return accuracy_rate

proc2 = P2(input['train_trial'], input['test_trial'])
proc2._pre_train_proc()
proc2._train_proc()
proc2._pre_test_proc()
acr = proc2._test_proc()
print(acr)

(8, 97) (8, 97, 97)
(97,) (97,) (97, 97)
0.0


LinAlgError: ignored

# Processor of model 3

In [36]:
class P3:
    def __init__(self, train, test):
        self.train = train
        self.test = test
        super().__init__()
    def _pre_train_proc(self):
        self.trials = self.train.shape[0]
        self.angles = self.train.shape[1]
        self.neurons = self.train[0][0][1].shape[0]
        self.beg = 351
        self.end = 551
    def _train_proc(self):
        self.mean = np.zeros((self.angles, self.neurons), dtype=np.float)
        for (trial_id, trial) in enumerate(self.train):
            for (angle_id, angle) in enumerate(trial):
                self.mean[angle_id] = self.mean[angle_id] + angle[1][:,self.beg:self.end].sum(1)
        self.mean = self.mean / self.trials
    def _pre_test_proc(self):
        self.trials = self.test.shape[0]
        self.angles = self.test.shape[1]
        self.neurons = self.test[0][0][1].shape[0]
        self.beg = 351
        self.end = 551
        print(self.trials, self.angles, self.neurons)
    def _test_proc(self):
        accuracy_cases = 0
        for (trial_id, trial) in enumerate(self.test):
            # print('trial: ', trial_id)
            for (angle_id, angle) in enumerate(trial):
                for (neuron_id, neuron) in enumerate(angle[1]):
                    # print(neuron.sum())
                    max_value = -10000
                    max_id = 0
                    for comp_id in range(self.angles):
                        temp = poisson_prob(neuron[self.beg:self.end].sum(), self.mean[comp_id][neuron_id])
                        if temp > max_value:
                            max_value = temp
                            max_id = comp_id
                    if max_id == angle_id:
                        accuracy_cases = accuracy_cases + 1
        accuracy_rate = accuracy_cases / (self.trials*self.angles*self.neurons)
        return accuracy_rate

proc3 = P3(input['train_trial'], input['test_trial'])
proc3._pre_train_proc()
proc3._train_proc()
proc3._pre_test_proc()
acr = proc3._test_proc()
print(acr)

91 8 97
0.19018352781239378
